## Установим необходимые зависимости
Перед этим, для дальнейшего удобства переноса, удалим начальные зависимости Google Colab'а

А нужные зависимости сохраним, чтобы можно было удобно использовать их на ВМ

In [ ]:
import subprocess
import os
from google.colab import drive
from tqdm import tqdm

# Подключаем Google Drive
drive.mount('/content/drive')

not_to_delete_packages = ["ipython", "ipykernel", "tornado", "prompt-toolkit", "pyzmq", "tqdm", "IPython", "pillow", "certifi", "dateutil", "decorator", "google", "matplotlib_inline" ,"pexpect", "psutil", "wcwidth", "pdf2image"]

# Указываем путь для дублирования requirements.txt в Google Drive
backup_path = '/content/drive/MyDrive/HacksAI/СЗФО24/requirements.txt'  # Измените путь по необходимости

# Проверяем, существует ли requirements.txt
if not os.path.exists('requirements.txt'):
    # Сохраняем текущие зависимости в old_requirements.txt
    !pip freeze > old_requirements.txt

    # Считываем old_requirements.txt
    with open('old_requirements.txt') as f:
        packages = f.read().splitlines()

    # Пытаемся удалить каждый пакет по очереди
    for package in tqdm(packages):
        # Если хотя бы одно имя из списка not_to_delete_packages входит в package, пропускаем этот пакет
        if any(n_package in package for n_package in not_to_delete_packages):
            continue
        try:
            if "google" in package:
              print("Скрипт не работает")

            subprocess.run(['pip', 'uninstall', '-y', package], check=True)
        except subprocess.CalledProcessError:
            print(f"Не удалось удалить {package}. Пропускаем...")


# Устанавливаем необходимые пакеты
!pip install markdown-it-py mdit_plain pypdf openpyxl python-docx python-pptx haystack-ai ragas-haystack vllm chroma-haystack boto3 PyPDF2
!pip install "datasets>=2.6.1" "sentence-transformers>=3.0.0"

# Сохраняем текущие зависимости в requirements.txt
!pip freeze > requirements.txt

#Дублируем файл requirements.txt в Google Drive
!cp requirements.txt {backup_path}

print(f"Файл requirements.txt успешно сохранен и скопирован в Google Drive по пути: {backup_path}")

Mounted at /content/drive


  6%|▌         | 30/538 [00:19<05:12,  1.63it/s]

Не удалось удалить blinker==1.4. Пропускаем...


100%|██████████| 538/538 [07:04<00:00,  1.27it/s]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 22.2

### Импорты

In [ ]:
from haystack import Pipeline, Document, component
from haystack.components.converters import PyPDFToDocument, DOCXToDocument, PPTXToDocument
from haystack.components.converters.csv import CSVToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.utils import ComponentDevice, Device, Secret
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever, InMemoryBM25Retriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage, ChatRole
from haystack.components.builders import ChatPromptBuilder

/usr/local/lib/python3.10/dist-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from pathlib import Path
from pprint import pprint

import yaml
import requests
from datetime import datetime, timezone
from PyPDF2 import PdfReader

import gc
import torch
import re
import json
from pydantic import BaseModel
from typing import List, Optional, Dict, Any, Union
from jinja2 import Template

import os
import time
import pandas as pd

# s3 client
import boto3
from botocore.exceptions import ClientError, NoCredentialsError

# pdf2image converter
# from pdf2image import convert_from_path
from PIL import Image
import logging

from google.colab import userdata

In [ ]:
if torch.cuda.is_available():
    device = ComponentDevice.from_single(Device.gpu(id=0))
else:
    device = ComponentDevice.from_single(Device.cpu())

### Переменные окружения

In [ ]:
base_congif_dir = '/content/drive/MyDrive/HacksAI/international24/config'

os.environ['PROMPTS_CONFIG_PATH'] = os.path.join(base_congif_dir, 'prompts.yaml')
os.environ['DICTS_CONFIG_PATH'] = os.path.join(base_congif_dir, 'dicts.yaml')
os.environ['RAG_CONFIG_PATH'] = os.path.join(base_congif_dir, 'rag.yaml')
os.environ['API_CONFIG_PATH'] = os.path.join(base_congif_dir, 'api.yaml')

os.environ['VLLM_URL'] = "http://localhost:8000/v1"
os.environ['CHROMA_HOSTNAME'] = "localhost"
os.environ['CHROMA_PORT'] = "8081"

In [ ]:
from google.colab import userdata

os.environ['MAIN_DOCS_DIR'] = "/content/drive/MyDrive/HacksAI/international24/dataset"
os.environ['API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['YANDEX_API_TOKEN'] = userdata.get('YANDEX_API_TOKEN')

In [ ]:
VLLM_URL = os.getenv("VLLM_URL")
CHROMA_HOSTNAME = os.getenv("CHROMA_HOSTNAME")
CHROMA_PORT = os.getenv("CHROMA_PORT")

YANDEX_API_TOKEN = os.getenv("YANDEX_API_TOKEN")

### Конфиги

In [ ]:
MAIN_DOCS_DIR = os.getenv('MAIN_DOCS_DIR')

# PROMPT CONFIG
PROMPTS_CONFIG_PATH = os.getenv("PROMPTS_CONFIG_PATH")
with open(PROMPTS_CONFIG_PATH, 'r', encoding='utf-8') as file:
    prompt_config = yaml.safe_load(file)

In [ ]:
# DICTS CONFIG
DICTS_CONFIG_PATH = os.getenv("DICTS_CONFIG_PATH")
with open(DICTS_CONFIG_PATH, 'r') as file:
    dicts_config = yaml.safe_load(file)

In [ ]:
# RAG CONFIG
RAG_CONFIG_PATH = os.getenv("RAG_CONFIG_PATH")
with open(RAG_CONFIG_PATH, "r") as file:
    rag_config = yaml.safe_load(file)

model = rag_config.get("model")
embedding_model = rag_config.get("embedding_model")

split_function_config = rag_config.get("split_function", {})
max_chunk_size = split_function_config.get("max_chunk_size", 300)
overlap = split_function_config.get("overlap", 0)

rag_gen_kwargs = rag_config.get("rag_gen_kwargs", {})
json_gen_kwargs = rag_config.get("json_gen_kwargs", {})

In [ ]:
# API CONFIG
API_CONFIG_PATH = os.getenv("API_CONFIG_PATH")
with open(API_CONFIG_PATH, "r", encoding='utf-8') as config_file:
  api_config = yaml.safe_load(config_file)

# Извлечение URL и эндпоинтов из конфигурации
BASE_URL = api_config["api"]["base_url"]
POLL_ENDPOINT = f"{BASE_URL}{api_config['api']['endpoints']['poll']}"
COMPLETE_ENDPOINT = f"{BASE_URL}{api_config['api']['endpoints']['complete']}"

### Кастомные обёртки компонент
#### Query Expander, который поможет расширить вопросы для поиска по Базе знаний

In [ ]:
@component
class QueryExpander:
    def __init__(
            self,
            system_prompt: str,
            user_prompt_template: str,
            json_gen_kwargs,
    ):
        self.system_prompt = system_prompt
        self.user_prompt_template = user_prompt_template

        builder = ChatPromptBuilder(variables=["query", "user_info"])
        llm = create_generator(json_gen_kwargs)

        self.pipeline = Pipeline()
        self.pipeline.add_component(name="builder", instance=builder)
        self.pipeline.add_component(name="llm", instance=llm)
        self.pipeline.connect("builder", "llm")


    @component.output_types(queries=List[str])
    def run(
        self,
        query: str,
        ):
        messages = [
            ChatMessage.from_system(self.system_prompt),
            ChatMessage.from_user(self.user_prompt_template)
        ]

        result = self.pipeline.run({
            'builder': {
                'template': messages,
                'query': query,
            }
        })

        response_text = result['llm']['replies'][0].content

        def extract_json_array(text):
            last_bracket_idx = text.rfind(']')
            if last_bracket_idx == -1:
                return None, text
            first_bracket_idx = text.rfind('[', 0, last_bracket_idx)
            if first_bracket_idx == -1:
                return None, text
            json_str = text[first_bracket_idx:last_bracket_idx+1]
            remaining_text = text[:first_bracket_idx].strip()
            return json_str, remaining_text

        json_str, remaining_text = extract_json_array(response_text)

        expanded_queries = []

        if json_str:
            try:
                expanded_queries = json.loads(json_str)

            except Exception as e:
              print(e)
              return {"queries": [query]}

        expanded_queries.append(query)

        return {"queries": expanded_queries}

#### Класс для Text Embedder'а, способный работать сразу с несколькими запросами (которые приходят после Query Expander'a)

In [ ]:
@component
class MultiQueryTextEmbedder:
    def __init__(self, embedder: SentenceTransformersTextEmbedder, top_k: int = 1):
        self.embedder = embedder
        self.embedder.warm_up()
        self.results = []
        self.ids = set()
        self.top_k = top_k

    @component.output_types(embeddings=List[List[str]])
    def run(self, queries: List[str]):
      self.results = []
      for query in queries:
        self.results.append(self.embedder.run(query))

      return {"embeddings": self.results}

#### Embedding Retriever, способный к векторному поиску по сразу нескольким эмбеддингам

In [ ]:
@component
class MultiQueryInMemoryRetriever:
    def __init__(self, retriever: InMemoryEmbeddingRetriever, filters=None, top_k: int = 3, score_threshold: float = 0.0):

        self.retriever = retriever
        self.results = []
        self.ids = set()
        self.top_k = top_k
        self.filters=filters
        self.score_threshold = score_threshold

    def add_document(self, document: Document):
        if (document.id not in self.ids) and (document.score > self.score_threshold):
            self.results.append(document)
            self.ids.add(document.id)

    @component.output_types(documents=List[Document])
    def run(self, emdeddings: List[List[str]], filters=None):
        self.results = []
        self.ids = set()

        for emdedding in emdeddings:
          result = self.retriever.run(query_embedding=emdedding['embedding'], filters=filters, top_k = self.top_k)
          for doc in result['documents']:
            self.add_document(doc)

        self.results.sort(key=lambda x: x.score, reverse=True)

        return {"documents": self.results}

#### BM25 Retriever, который делает поиск по ключевым словам сразу по нескольким вопросам

In [ ]:
@component
class MultiQueryInMemoryBM25Retriever:
    def __init__(self, retriever: InMemoryBM25Retriever, top_k: int = 3, filters=None):

        self.retriever = retriever
        self.results = []
        self.ids = set()
        self.filters=filters
        self.top_k = top_k

    def add_document(self, document: Document):
        if document.id not in self.ids:
            self.results.append(document)
            self.ids.add(document.id)

    @component.output_types(documents=List[Document])
    def run(self, queries: List[str], top_k: int = None, filters=None):
        self.results = []
        self.ids = set()
        if top_k != None:
          self.top_k = top_k
        for query in queries:
          result = self.retriever.run(query = query, filters = filters, top_k = self.top_k)
          for doc in result['documents']:
            self.add_document(doc)
        self.results.sort(key=lambda x: x.score, reverse=True)
        return {"documents": self.results}

## Создадим пайплайн для индексации документов

In [ ]:
def create_in_memory_document_store():
  document_store = InMemoryDocumentStore()

  return document_store

def create_chroma_document_store():
  document_store = ChromaDocumentStore(
      host = CHROMA_HOSTNAME,
      port = CHROMA_PORT
  )

  return document_store

def create_indexing_pipeline(document_store):

  file_type_router = FileTypeRouter(mime_types=[
      "application/pdf",
      "text/csv",
      "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
      "application/vnd.openxmlformats-officedocument.presentationml.presentation",
      ])

  pdf_converter = PyPDFToDocument()
  docx_converter = DOCXToDocument()
  csv_converter = CSVToDocument()
  pptx_converter = PPTXToDocument()

  document_joiner = DocumentJoiner()
  document_cleaner = DocumentCleaner()
  document_splitter = DocumentSplitter(split_by="page", split_length=1, split_overlap=0)
  document_writer = DocumentWriter(document_store)
  document_embedder = SentenceTransformersDocumentEmbedder(model = embedding_model, device = device)

  indexing_pipeline = Pipeline()

  indexing_pipeline.add_component(instance=file_type_router, name="file_type_router")
  indexing_pipeline.add_component(instance=pdf_converter, name="pypdf_converter")
  indexing_pipeline.add_component(instance=docx_converter, name="docx_converter")
  indexing_pipeline.add_component(instance=csv_converter, name="csv_converter")
  indexing_pipeline.add_component(instance=pptx_converter, name="pptx_converter")

  indexing_pipeline.add_component(instance=document_joiner, name="document_joiner")
  indexing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
  indexing_pipeline.add_component(instance=document_splitter, name="document_splitter")
  indexing_pipeline.add_component(instance=document_embedder, name="document_embedder")
  indexing_pipeline.add_component(instance=document_writer, name="document_writer")

  indexing_pipeline.connect("file_type_router.application/pdf", "pypdf_converter.sources")
  indexing_pipeline.connect(
      "file_type_router.application/vnd.openxmlformats-officedocument.wordprocessingml.document",
      "docx_converter.sources")
  indexing_pipeline.connect(
      "file_type_router.application/vnd.openxmlformats-officedocument.presentationml.presentation",
      "pptx_converter.sources"
  )
  indexing_pipeline.connect("file_type_router.text/csv", "csv_converter.sources")

  indexing_pipeline.connect("pypdf_converter", "document_joiner")
  indexing_pipeline.connect("docx_converter", "document_joiner")
  indexing_pipeline.connect("csv_converter", "document_joiner")
  indexing_pipeline.connect("pptx_converter", "document_joiner")

  indexing_pipeline.connect("document_joiner", "document_splitter")
  indexing_pipeline.connect("document_splitter", "document_cleaner")
  indexing_pipeline.connect("document_cleaner", "document_embedder")
  indexing_pipeline.connect("document_embedder", "document_writer")

  return indexing_pipeline


document_store = create_in_memory_document_store()

indexing_pipeline = create_indexing_pipeline(document_store = document_store)

### Запустит VLLM Сервер, с OpenAI Like API

In [ ]:
def create_generator(gen_kwargs = None):
  return OpenAIChatGenerator(
          api_key = Secret.from_token("VLLM-PLACEHOLDER-API-KEY"),
          model = model,
          api_base_url = VLLM_URL,
          generation_kwargs = gen_kwargs,
          timeout = 600,
        )

In [ ]:
! nohup python -m vllm.entrypoints.openai.api_server \
                  --model {model} \
                  --max-model-len 32000 \
                  --dtype half \
                  --quantization gptq \
                  --gpu_memory_utilization 0.75 \
                  > vllm.log &

nohup: redirecting stderr to stdout


In [ ]:
!while ! grep -q "Application startup complete" vllm.log; do tail -n 1 vllm.log; sleep 5; done

INFO 11-10 05:39:44 api_server.py:179] Started engine process with PID 10108
WARNING 11-10 05:39:50 arg_utils.py:1019] [DEPRECATED] Block manager v1 has been removed, and setting --use-v2-block-manager to True or False has no effect on vLLM behavior. Please remove --use-v2-block-manager in your engine argument. If your use case is not supported by SelfAttnBlockSpaceManager (i.e. block manager v2), please file an issue with detailed information.
INFO 11-10 05:39:54 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_red

### Индексация
#### Пайплайн, который при индексации делает суммаризацию (заголовок, summary и категории)

In [ ]:
def create_summarize_pipeline():
  pipeline = Pipeline()

  prompt_builder = ChatPromptBuilder(variables = ['document_fragment'])

  generator = create_generator(rag_gen_kwargs)

  pipeline.add_component("prompt_builder", prompt_builder)
  pipeline.add_component("generator", generator)

  pipeline.connect("prompt_builder", "generator")

  return pipeline

In [ ]:
summarize_pipeline = create_summarize_pipeline()

gen_questions_system_message = ChatMessage.from_system(prompt_config['summarize_system_prompt'])
summarize_messages = [gen_questions_system_message]

In [ ]:
def generate_summarization(document_content):
    # Выполняем запрос к пайплайну суммаризации
    response = summarize_pipeline.run({"prompt_builder": {"template": summarize_messages, "document_fragment": document_content}})

    # Извлекаем текст ответа
    response_text = response['generator']['replies'][0].content

    # Поиск первого и последнего вхождения фигурных скобок для выделения JSON
    json_str = None
    try:
        first_brace = response_text.find('{')
        last_brace = response_text.rfind('}')

        # Если фигурные скобки найдены, извлекаем текст между ними
        if first_brace != -1 and last_brace != -1:
            json_str = response_text[first_brace:last_brace + 1]
            response_dict = json.loads(json_str)
        else:
            print("Error: JSON structure not found in response.")
            return {}

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        print(f"Attempted JSON string: {json_str}")
        return {}

    return response_dict

Для суммаризации берём несколько первых страниц

In [ ]:
def extract_context_from_pdf(pdf_path: str) -> str:
    context = ""
    try:
        # Read PDF content using PyPDF2
        with open(pdf_path, 'rb') as f:
            reader = PdfReader(f)
            # Set the number of pages to extract (up to 3)
            num_pages_to_extract = min(7, len(reader.pages))

            # Extract text from each page up to the third page
            for page_num in range(num_pages_to_extract):
                page = reader.pages[page_num]
                text = page.extract_text()
                if text:
                    context += text

            # Return the extracted context or a fallback if empty
            return context.strip() if context else "No text extracted from the first three pages."

    except Exception as e:
        print(f"An error occurred while extracting context from '{pdf_path}': {e}")
        return 'Error extracting context'

In [ ]:
# test_extracted_pdf = extract_context_from_pdf("drive/MyDrive/HacksAI/international24/dataset/Media_Digital/presentations/1.pdf")
# test_summ = generate_summarization(test_extracted_pdf)
# print(test_summ)

#### Функции для полной переиндексации и добавления какого-то одного документа

In [ ]:
def do_reindex():
    orgs_dirs = Path(MAIN_DOCS_DIR)
    for org_dir in orgs_dirs.iterdir():
        if org_dir.is_dir():
            source_docs = list(org_dir.glob("**/*"))
            for doc in source_docs:
                # Проверка типа файла и обработка PDF
                if doc.suffix.lower() == ".pdf":
                    # Извлечение контекста из PDF и генерация суммаризации
                    context = extract_context_from_pdf(doc)
                    summarization = generate_summarization(context)

                    # Заполнение метаданных из суммаризации
                    meta_data = {
                        "space": org_dir.name,
                        "filename": os.path.basename(doc),
                        "title": summarization.get("title", ""),
                        "summary": summarization.get("summary", ""),
                        "categories": summarization.get("categories", [])
                    }
                else:
                    # Для других типов файлов - заполняем метаданные пустыми значениями
                    meta_data = {
                        "space": org_dir.name,
                        "filename": os.path.basename(doc),
                        "title": "",
                        "summary": "",
                        "categories": []
                    }

                # Запуск пайплайна индексации с заполненными метаданными
                indexing_pipeline.run({
                    "file_type_router": {"sources": [doc]},
                    "pypdf_converter": {"meta": meta_data},
                    "csv_converter": {"meta": meta_data},
                    "docx_converter": {"meta": meta_data},
                    "pptx_converter": {"meta": meta_data},
                })

                # Очистка памяти
                gc.collect()
                torch.cuda.empty_cache()


def do_index_single_file(file_path):
    # Преобразуем путь к файлу в объект Path
    file_path = Path(file_path)

    # Проверяем, что файл существует
    if not file_path.exists() or not file_path.is_file():
        raise ValueError(f"The file {file_path} does not exist or is not a valid file.")

    # Извлекаем директорию, в которой находится файл, для использования в метаданных
    space = file_path.parent.name
    filename = os.path.basename(file_path)

    # Проверка типа файла и обработка PDF
    if file_path.suffix.lower() == ".pdf":
        # Извлечение контекста из PDF и генерация суммаризации
        context = extract_context_from_pdf(file_path)
        summarization = generate_summarization(context)

        # Заполнение метаданных из суммаризации
        meta_data = {
            "space": space,
            "filename": filename,
            "title": summarization.get("title", ""),
            "summary": summarization.get("summary", ""),
            "categories": summarization.get("categories", [])
        }
    else:
        # Для других типов файлов - заполняем метаданные пустыми значениями
        meta_data = {
            "space": space,
            "filename": filename,
            "title": "",
            "summary": "",
            "categories": []
        }

    # Запуск пайплайна индексации для одного файла с заполненными метаданными
    indexing_pipeline.run({
        "file_type_router": {"sources": [file_path]},
        "pypdf_converter": {"meta": meta_data},
        "csv_converter": {"meta": meta_data},
        "docx_converter": {"meta": meta_data},
        "pptx_converter": {"meta": meta_data},
    })

In [ ]:
# do_reindex()

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.81it/s]
[0, IndirectObject(7279, 0, 138400495375728), 1, 2, 0, 3, 5, 249, 6, IndirectObject(7280, 0, 138400495375728), 21, 30, 574, 31, IndirectObject(7281, 0, 138400495375728), 116, 117, 373, 118, IndirectObject(7282, 0, 138400495375728), 203, 204, 653, 205, IndirectObject(7283, 0, 138400495375728), 262, 263, 587, 264, IndirectObject(7284, 0, 138400495375728), 267, 268, 623, 269, IndirectObject(7285, 0, 138400495375728), 282, 284, 560, 285, IndirectObject(7286, 0, 138400495375728), 292, 294, 866, 295, IndirectObject(7287, 0, 138400495375728), 301, 302, 365, 303, IndirectObject(7288, 0, 138400495375728), 304, 305, 365, 306, 307, 324, 308, 309, 560, 310, IndirectObject(7289, 0, 138400495375728), 311, 312, 413, 313, IndirectObject(7290, 0, 138400495375728), 328, 331, 521, 332, IndirectObject(7291, 0, 138400495375728), 354, 355, 432, 356, IndirectObject(7292, 0, 138400495375728), 357, 358, 287, 359, IndirectObject(7293, 0, 138400495375728), 3

Error: JSON structure not found in response.


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.61it/s]


Error: JSON structure not found in response.


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.37it/s]


...

Batches: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]


Error parsing JSON response: Unterminated string starting at: line 1 column 2 (char 1)
Attempted JSON string: {"ьькиььььькикиеьььььки 5ькикикикикиькикикиььььки степеникикикикикиьььььькиьььькитокиькикиоль73ена 1171ькикиькиущкиььки рядььь_23ььольььяькиьььеляььеськиькиьььььььвлньневькиькиьььки_20ььькикикикикикиььььььькикиььькикиььр3ьnbsp2яьььььььььььььььььььььакььььасььььььькикикикиьькитькикикикикикикикикикикикикикиььььольноневкиьькикикикикиьдакиьькикикики 1ник................................31киььььькиькикиььььььььькиьькиььемьььььькиеьььььььььь...... дивььььььььникьькаькикикиьььькиььььькиььькаьььья такьььььььоль6олььььььььььькиьььььькикиььствоьььькаькиььььякикиь 5ььььььькикиьььььчиьькикикикиьчикаьььььнев1родььььььььььькиьььььььььькиьььььььольььььькикикиьььььььььььььььььььькикикикикиьььььвьькиениькикикикикикиениььььььььькиьки 1ольььььььььььььььььььььььольььькиьольолььолььневььььььворникьькикикиродяьькиькиольжольолььььььоль...1_m-------------------------------------------------------------

Batches: 100%|██████████| 1/1 [00:00<00:00, 13.51it/s]


Error parsing JSON response: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
Attempted JSON string: {
{" {" {}
{".

{"инавассассассюассс {" {" нель limburg У {" {" {" utrechtuspendLayoutщведенияведенияассс {" {" {" {" {" {" utrechtщщ-thumbnails {" limburg {"
{" текстассасс-INFRINGEMENTассасасснь {}
{" censorship {" {" {" {" {" {" utrechtструкториню У {" {" {"асс {" {" utrecht INFORMATIONассегь {}
{" {"{"ставстав {" {" {" {" {" {" {" {" {"族自治县щассассм {"веденияассю{" {" {" {" {"ассисchluss örвасс� {"щассмассюассассведенияассendcode {"endcodeщщщщ興ставInChildrençasщ
{"�веденияинубщщ {" {" {" {" {" {" {"ехведения {" {" {" {"� {"endcode {}
{"�инекинассассегениасс {" {" {" {" {" {" {" {" {"endcodeyntaxException� {" {" {" {" {" {" {" {"chriftставставс {"ставInChildrenс {" {" {" {" {" {" {" {"-commerceум {" {" {" {" {" {" {" {" {" {" {" {" {" {" {"{" {"族自治县-INFRINGEMENTendcode {" {" {"щ {" {" {" {" {" {" {" {" {" {" {" {" {"chriftщ {"ж {" {"ведения документщставинег

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


## Собираем RAG пайплайн

### Инициализируем все части пайплайна:

*   Expander (перефразирование вопроса для удобного поиска)
*   Text Embedder (векторные представления вопросов)
*   Гибридный поиск - два retriever'a (векторный поиск и по ключевым словам)
*   Prompt Builder (построение структуры промпта)
*   Generator (модель для генерации ответа на основе контекста)






In [ ]:
def create_rag_pipeline(
    document_store,
    ) -> Pipeline:
  expander = QueryExpander(
      system_prompt = prompt_config['query_expander_system_prompt'],
      user_prompt_template = prompt_config['query_expander_user_prompt_template'],
      json_gen_kwargs = json_gen_kwargs,
  )
  document_joiner = DocumentJoiner(top_k = rag_config.get("document_joiner").get("top_k", 4), join_mode = "distribution_based_rank_fusion")
  text_embedder = MultiQueryTextEmbedder(SentenceTransformersTextEmbedder(model=embedding_model, device=device))
  embedding_retriever = MultiQueryInMemoryRetriever(InMemoryEmbeddingRetriever(document_store), top_k = rag_config.get("embedding_retriever").get("top_k", 3))
  bm25_retriever = MultiQueryInMemoryBM25Retriever(InMemoryBM25Retriever(document_store), top_k = rag_config.get("bm25_retriever").get("top_k", 3))

  final_answer_prompt_builder = ChatPromptBuilder(variables=["documents", "question"])

  doc_relevant_generator = create_generator(rag_gen_kwargs)
  final_answer_generator = create_generator(rag_gen_kwargs)

  rag_pipeline = Pipeline()

  rag_pipeline.add_component("expander", expander)
  rag_pipeline.add_component("text_embedder", text_embedder)
  rag_pipeline.add_component("embedding_retriever", embedding_retriever)
  rag_pipeline.add_component("bm25_retriever", bm25_retriever)
  rag_pipeline.add_component("document_joiner", document_joiner)

  rag_pipeline.add_component("final_answer_prompt_builder", final_answer_prompt_builder)
  rag_pipeline.add_component("final_answer_llm", final_answer_generator)


  rag_pipeline.connect("expander.queries", "text_embedder.queries")
  rag_pipeline.connect("expander.queries", "bm25_retriever.queries")
  rag_pipeline.connect("text_embedder.embeddings", "embedding_retriever")
  rag_pipeline.connect("bm25_retriever", "document_joiner")
  rag_pipeline.connect("embedding_retriever", "document_joiner")

  rag_pipeline.connect("document_joiner", "final_answer_prompt_builder.documents")
  rag_pipeline.connect("final_answer_prompt_builder", "final_answer_llm")

  return rag_pipeline

In [ ]:
rag_pipeline = create_rag_pipeline(document_store = document_store)

In [ ]:
chat_system_message = ChatMessage.from_system(prompt_config['qwen_chat_system_prompt'])
chat_user_message = ChatMessage.from_user(prompt_config['qwen_chat_user_prompt_template'])

final_answer_messages = [chat_system_message, chat_user_message]

### Функция для обычного ответа на вопрос

In [ ]:
class Reference(BaseModel):
    filename: str
    page_number: str

class ModelResponse(BaseModel):
    answer: str
    references: Optional[List[Reference]] = None

def get_chat_response(
    question: str,
    categories: Optional[List[str]] = None,
    space: Optional[str] = None,
    filename: Optional[str] = None,
) -> ModelResponse:
    # Формирование фильтров для bm25_retriever и embedding_retriever
    filters = {"operator": "AND", "conditions": []}
    if categories:
        filters["conditions"].append({"field": "meta.categories", "operator": "in", "value": categories})
    if space:
        filters["conditions"].append({"field": "meta.space", "operator": "==", "value": space})
    if filename:
        filters["conditions"].append({"field": "meta.filename", "operator": "==", "value": filename})
    # Запускаем RAG-пайплайн с фильтрами
    response = rag_pipeline.run({
        "expander": {
            "query": question,
        },
        "bm25_retriever": {
            "filters": filters,
        },
        "embedding_retriever": {
            "filters": filters,
        },
        "final_answer_prompt_builder": {
            "question": question,
            "template": final_answer_messages,
        },
    }, include_outputs_from={"expander", "document_joiner", "final_answer_prompt_builder"})

    response_text = response['final_answer_llm']['replies'][0].content

    references = None
    try:
        # Находим индекс последнего символа ']'
        last_bracket_idx = response_text.rfind(']')
        if last_bracket_idx != -1:
            # Находим индекс соответствующего символа '[' перед ']'
            first_bracket_idx = response_text.rfind('[', 0, last_bracket_idx)
            if first_bracket_idx != -1:
                json_str = response_text[first_bracket_idx:last_bracket_idx+1]

                try:
                    # Парсим JSON-строку
                    references_list = json.loads(json_str)
                    references = [Reference(**item) for item in references_list]
                    response_text = response_text[:first_bracket_idx].strip()
                except json.JSONDecodeError as e:
                    print(f"Ошибка при парсинге JSON: {e}")
    except Exception as e:
        print(f"Общая ошибка при обработке ответа: {e}")

    gc.collect()
    torch.cuda.empty_cache()

    return ModelResponse(answer=response_text, references=references)

In [ ]:
test_response = get_chat_response(
    question = "Какой процент выручки от рекламы достигала Meta в 2020 году?"
    )
pprint(test_response.answer)
print(test_response.references)

Batches: 100%|██████████| 1/1 [00:00<00:00, 59.92it/s]


('В 2020 году доля выручки от рекламы Meta, которой принадлежат Facebook и '
 'Instagram, достигала 98%.')
[Reference(filename='1.pdf', page_number='7')]


### Функция для JSON'а на Front-End

In [ ]:
def model_response_to_json(
    model_response: ModelResponse,
    categories: Optional[List[str]] = None,
    space: Optional[str] = None
) -> Dict[str, Any]:
    sources = []

    if model_response.references:
        for ref in model_response.references:
            # Фильтруем только по page_number и filename
            filters = {
                "operator": "AND",
                "conditions": [
                    {"field": "meta.page_number", "operator": "==", "value": int(ref.page_number)},
                    {"field": "meta.filename", "operator": "==", "value": ref.filename},
                ]
            }

            # Извлечение единственного документа из document_store по фильтрам
            documents = document_store.filter_documents(filters=filters)
            document = documents[0] if documents else None  # Получаем первый документ или None, если документов нет

            if document and document.meta.get("filename") == ref.filename:
                # Извлечение необходимых полей
                slide_snippet = document.content
                filename = document.meta.get("filename")
                slide_number = document.meta.get("page_number")

                # Извлечение названия презентации из имени файла (без расширения)
                file_title = filename.rsplit('.', 1)[0] if filename else "Unknown"

                # Формирование метаданных с использованием space и categories
                metadata = {
                    "title": f"{file_title} Слайд №{slide_number}",
                    "file_title": file_title,
                    "file_name": filename,
                    "file_category": categories if categories else [None],  # Используем categories или [null] в JSON
                    "file_space": space,       # Используем space или null в JSON
                    "slide_number": slide_number,
                    "url": f"https://storage.yandexcloud.net/hacks-ai-storage/{filename}",
                    "thumbnail": f"https://storage.yandexcloud.net/hacks-ai-storage/thumb_{filename}_{slide_number}.png"
                }

                # Формирование источника
                source = {
                    "pageContent": slide_snippet,
                    "metadata": metadata
                }

                sources.append(source)
            else:
                # Обработка случая, когда документ не найден
                print(f"Документ не найден для filename: {ref.filename}, page_number: {ref.page_number}")

    # Формирование итогового JSON
    json_output = {
        "message": model_response.answer,
        "sources": sources
    }

    return json_output

In [ ]:
model_response_to_json(test_response)

{'message': 'В 2020 году доля выручки от рекламы Meta, которой принадлежат Facebook и Instagram, достигала 98%.',
 'sources': [{'pageContent': 'Коммуникационные сервисы: социальные сети и мессенджеры\nДля сравнения, только 23% респондентов из Японии ориентируются на социальные сети как на первоисточник. В России мессенджеры активно используются для профес-\nсиональных коммуникаций. 7 По результатам опроса, приблизительно 70% работающих россиян постоянно обсуждают рабочие вопросы в мессенджерах. При этом традиционные способы коммуникаций по электронной почте и в мессенджерах остаются одинаково предпочтительными Стоимость за тысячу показов рекламы в социальных сетях составила 5,33 долл. во втором квартале 2023 года, что на 2 долл. меньше, чем в предыдущем году. Эксперты отмечают, что пассивные виды В качестве основной бизнес-модели социальные сети используют получение выручки от рекламных услуг и внедряемую с недавнего времени подписную модель. Маркетологи выделяют Facebook* как самую по

In [ ]:
def get_chat_response_json(input_data: Dict[str, Any]) -> str:
    # Извлечение значений из входного JSON-данных
    categories = input_data.get("categories", None)
    space = input_data.get("space", None)
    filename = input_data.get("filename", None)
    question = input_data.get("question", "")

    # Получаем ответ модели с учетом извлеченных параметров
    model_response = get_chat_response(
        question=question,
        categories=categories,
        space=space,
        filename=filename
    )

    # Преобразуем ответ модели в JSON с передачей space и categories
    json_output = model_response_to_json(model_response, categories=categories, space=space)

    return json_output

### Для генерации Related вопросов
которые подскажут пользователю возможные подходящие вопросы

In [ ]:
def create_gen_rel_questions_pipeline():
  pipeline = Pipeline()

  prompt_builder = ChatPromptBuilder()
  generator = create_generator() # Можно параметры чтобы поразнообразнее

  pipeline.add_component("prompt_builder", prompt_builder)
  pipeline.add_component("generator", generator)

  pipeline.connect("prompt_builder", "generator")

  return pipeline

In [ ]:
gen_questions_pipeline = create_gen_rel_questions_pipeline()

gen_questions_system_message = ChatMessage.from_system(prompt_config['qwen_gen_related_system_prompt'])

In [ ]:
def generate_related_questions(conversational: List[List[Union[str, str]]]) -> List[str]:
    # Инициализируем сообщения, добавляя системное сообщение
    chat_messages = [gen_questions_system_message]

    # Фильтруем и ограничиваем историю переписки до последних 5 сообщений
    filtered_conversational = conversational[-5:]

    # Формируем сообщения на основе фильтрованных данных
    for role, content in filtered_conversational:
        if role in {"human", "user"}:
            chat_messages.append(ChatMessage.from_user(content))
        else:  # Любая другая роль трактуется как сообщение ассистента
            chat_messages.append(ChatMessage.from_assistant(content))

    # Запуск пайплайна с подготовленными сообщениями
    result = gen_questions_pipeline.run({"prompt_builder": {"template": chat_messages}})

    # Извлечение и парсинг сгенерированного ответа
    generated_replies = result["generator"]["replies"][0].content
    questions = []
    try:
        # Находим индекс последнего символа ']' и первого '[' перед ним
        last_bracket_idx = generated_replies.rfind(']')
        if last_bracket_idx != -1:
            first_bracket_idx = generated_replies.rfind('[', 0, last_bracket_idx)
            if first_bracket_idx != -1:
                json_str = generated_replies[first_bracket_idx:last_bracket_idx + 1]

                try:
                    # Парсим JSON-строку как список вопросов
                    questions_list = json.loads(json_str)
                    if isinstance(questions_list, list) and all(isinstance(item, str) for item in questions_list):
                        questions = questions_list
                except json.JSONDecodeError as e:
                    print(f"JSON parsing error: {e}")
    except Exception as e:
        print(f"General error processing response: {e}")

    # Возвращаем распарсенные вопросы или пустой список при ошибке
    return questions

In [ ]:
conversational_example_1 = [
    ['human', "Привет! Можешь объяснить основные рекомендации по этому документу?"],
    ['assist', "Конечно! Документ описывает правила для сбалансированного питания и здорового образа жизни."],
    ['human', "Какие разделы охватывают советы по диете?"],
    ['assist', "Основные советы по диете даны в разделах 'Основы питания' и 'Рекомендуемые продукты'."],
    ['human', "Есть ли указания по планированию приёмов пищи?"],
]

conversational_example_2 = [
    ['human', "Здравствуйте! Я хочу понять, как использовать этот документ для изучения программирования."],
    ['assist', "Документ предлагает инструкции для начала работы с Python и JavaScript."],
    ['human', "Какие главы посоветуете для начинающих?"],
    ['assist', "В разделе 'Введение в программирование' и 'Основы Python' есть полезные рекомендации для начинающих."],
    ['human', "Документ упоминает навыки, требуемые для работы с данными?"],
]

conversational_example_3 = [
    ['human', "Hi! Can you help me understand the main goals outlined in this document?"],
    ['assist', "Sure! The document focuses on key strategies for project management and efficient teamwork."],
    ['human', "Which sections cover best practices for managing timelines?"],
    ['assist', "Best practices for timelines are discussed in the sections 'Time Management' and 'Project Scheduling'."],
    ['human', "Are there any specific tools recommended for project tracking?"],
]

In [ ]:
generate_related_questions(conversational_example_3)

['Are there any specific tools recommended for project tracking?',
 'Could you summarize the key points about team collaboration mentioned in the document?',
 'What are the main challenges in project management highlighted in the document?',
 'Are there any case studies or examples provided in the document?',
 'How does the document suggest handling unexpected delays or issues in a project?']

### Submission

In [ ]:
!ls drive/MyDrive/HacksAI/international24/dataset/Media_Digital/presentations/

0.pdf	15.pdf	20.pdf	26.pdf	31.pdf	37.pdf	42.pdf	48.pdf	53.pdf	59.pdf	64.pdf	6.pdf
10.pdf	16.pdf	21.pdf	27.pdf	32.pdf	38.pdf	43.pdf	49.pdf	54.pdf	5.pdf	65.pdf	70.pdf
11.pdf	17.pdf	22.pdf	28.pdf	33.pdf	39.pdf	44.pdf	4.pdf	55.pdf	60.pdf	66.pdf	71.pdf
12.pdf	18.pdf	23.pdf	29.pdf	34.pdf	3.pdf	45.pdf	50.pdf	56.pdf	61.pdf	67.pdf	7.pdf
13.pdf	19.pdf	24.pdf	2.pdf	35.pdf	40.pdf	46.pdf	51.pdf	57.pdf	62.pdf	68.pdf	8.pdf
14.pdf	1.pdf	25.pdf	30.pdf	36.pdf	41.pdf	47.pdf	52.pdf	58.pdf	63.pdf	69.pdf	9.pdf


In [ ]:
new_file_1 = "drive/MyDrive/HacksAI/international24/dataset/Media_Digital/presentations/70.pdf"
new_file_2 = "drive/MyDrive/HacksAI/international24/dataset/Media_Digital/presentations/71.pdf"

In [ ]:
# do_index_single_file(new_file_1)
# do_index_single_file(new_file_2)

Batches: 100%|██████████| 2/2 [00:00<00:00,  2.55it/s]


In [ ]:
def generate_submission_csv(input_csv: str, output_csv: str):
    """
    Reads questions from a CSV file, processes each question using get_chat_response,
    and writes the results to a CSV file.

    Parameters:
    - input_csv (str): Path to the input CSV file (questions.csv).
    - output_csv (str): Path to the output CSV file (submission.csv).
    """
    # Чтение CSV-файла в DataFrame
    try:
        df = pd.read_csv(input_csv, dtype={'question': str})
    except Exception as e:
        print(f"Ошибка при чтении файла CSV: {e}")
        return

    # Проверка наличия столбца questions
    if 'question' not in df.columns:
        print(df.columns)
        print("Отсутствует необходимый столбец: questions")
        return

    # Переименование столбца и добавление необходимых пустых столбцов
    df.rename(columns={'questions': 'question'}, inplace=True)
    df['answer'] = ""
    df['filename'] = ""
    df['slide_number'] = ""

    # Итерация по каждой строке и обработка вопроса
    for index, row in df.iterrows():
        question = row['question']
        print(question)
        if pd.isna(question) or not isinstance(question, str) or not question.strip().endswith('?'):
            # Если вход не является валидным вопросом

            df.at[index, 'filename'] = ""
            df.at[index, 'slide_number'] = ""
            df.at[index, 'answer'] = "Я не знаю ответа на ваш вопрос"
            continue

        # Получение ответа от модели
        model_response = get_chat_response(question)


        # Заполнение полей filename и slide_number без дублирования
        if model_response.references:
            # Извлечение имен файлов и номеров страниц
            filenames = [ref.filename for ref in model_response.references]
            slide_numbers = [ref.page_number for ref in model_response.references]

            filename = os.path.splitext(filenames[0])[0]
            slide_number = slide_numbers[0]

            # Объединение уникальных значений в строку через запятую
            df.at[index, 'filename'] = filename
            df.at[index, 'slide_number'] = slide_number
        else:
            # Если ссылки не найдены
            df.at[index, 'filename'] = ""
            df.at[index, 'slide_number'] = ""

        df.at[index, 'answer'] = model_response.answer

    # Запись обновленного DataFrame в CSV-файл
    try:
        df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"Файл успешно сохранен как {output_csv}")
    except Exception as e:
        print(f"Ошибка при записи файла CSV: {e}")

In [63]:
test_subm = "drive/MyDrive/HacksAI/international24/test.csv"

generate_submission_csv(
    input_csv = test_subm,
    output_csv = "submission.csv"
    )

Чем занимается платформа MEDIADESK?


Batches: 100%|██████████| 1/1 [00:00<00:00, 59.28it/s]


Чем отличаются возможности слабого (NARROW AI), сильного (AI) и супер-ИИ (SUPER AI)?


Batches: 100%|██████████| 1/1 [00:00<00:00, 61.29it/s]


Сколько процентов рутинных процессов отдаются искусственному интеллекту?


...

В каких аспектах рассматривается 3 эпоха интернета?


Batches: 100%|██████████| 1/1 [00:00<00:00, 58.59it/s]


Какие сервисы наиболее популярны в Великобритании в области соцсетей, поиска и торговли,?


Batches: 100%|██████████| 1/1 [00:00<00:00, 57.15it/s]


Файл успешно сохранен как submission.csv


In [64]:
test_subm_csv = pd.read_csv("submission.csv")
test_subm_csv

,question,answer,filename,slide_number
0,Чем занимается платформа MEDIADESK?,Я не знаю ответа на ваш вопрос,NaN,NaN
1,Чем отличаются возможности слабого (NARROW AI)...,Вот как отличаются возможности различных типов...,0.0,4.0
2,Сколько процентов рутинных процессов отдаются ...,80% рутинных процессов отдаются искусственному...,0.0,13.0
3,Как усложняться механики кампаний при использо...,Я не знаю ответа на ваш вопрос,NaN,NaN
4,На кого чаще всего подписаны пользователи соци...,Пользователи социальных сетей чаще всего подпи...,29.0,9.0
...,...,...,...,...
183,"Какие основные причины того, что контент на ви...","Основные причины, по которым контент на видеоп...",25.0,267.0
184,Сколько в процентах составляет доля людей по в...,По данным Reuters Institute for the Study of J...,59.0,17.0
185,Сколько процентов пользователей интернета в во...,"Исследования показывают, что около 60% пользов...",4.0,16.0
186,В каких аспектах рассматривается 3 эпоха интер...,Я не знаю ответа на ваш вопрос,NaN,NaN


### Evaluate
#### сохраним здесь ответы и контексты модели, затем в другом (evaluate) ноутбуке посчитаем различные метрики

In [ ]:
def get_eval_response(
    question: str,
    ):
    response = rag_pipeline.run({
        "expander": {
            "query": question,
            },
        "final_answer_prompt_builder": {
            "question": question,
            "template": final_answer_messages,
        },
    }, include_outputs_from={"expander", "document_joiner", "final_answer_prompt_builder"})

    response_text = response['final_answer_llm']['replies'][0].content

    references = None
    try:
        # Находим индекс последнего символа ']'
        last_bracket_idx = response_text.rfind(']')
        if last_bracket_idx != -1:
            # Находим индекс соответствующего символа '[' перед ']'
            first_bracket_idx = response_text.rfind('[', 0, last_bracket_idx)
            if first_bracket_idx != -1:
                json_str = response_text[first_bracket_idx:last_bracket_idx+1]

                try:
                    # Парсим JSON-строку
                    references_list = json.loads(json_str)
                    references = [Reference(**item) for item in references_list]
                    response_text = response_text[:first_bracket_idx].strip()
                except json.JSONDecodeError as e:
                    print(f"Ошибка при парсинге JSON: {e}")
    except Exception as e:
        print(f"Общая ошибка при обработке ответа: {e}")

    gc.collect()
    torch.cuda.empty_cache()
    try:
      r_slide_number = references[0].page_number if references else None
      r_filename = references[0].filename if references else None
    except Exception as e:
      r_slide_number = None
      r_filename = None

    contexts = []
    for doc in response['document_joiner']['documents']:
      contexts.append(doc.content)

    res_dict = {
        "response": response_text,
        "r_filename": r_filename,
        "r_slide_number": r_slide_number,
        "contexts": contexts,
    }

    return res_dict

In [ ]:
def add_rag_responses_to_csv(
    input_csv="ground_truth_data.csv",
    output_csv="gt_evaluation.csv",
    get_response_function=None,
    save_dir="",
    sub_dir_name=None  # Новый параметр для имени поддиректории
):
    """
    Читает CSV-файл с документами и вопросами, получает ответы с помощью RAG пайплайна,
    добавляет их в DataFrame и сохраняет результат в новый CSV-файл в указанной поддиректории с добавлением datetime.

    Параметры:
    - input_csv (str): Путь к входному файлу ground_truth_data.csv.
    - output_csv (str): Имя выходного файла с добавленными ответами.
    - get_response_function (callable): Функция для получения ответа из RAG пайплайна.
    - save_dir (str): Директория для сохранения выходного файла.
    - sub_dir_name (str, optional): Имя поддиректории внутри save_dir для сохранения файла. По умолчанию None.
    """

    if get_response_function is None:
        raise ValueError("Функция get_response_function должна быть передана и не должна быть None.")

    # Полный путь к входному файлу
    input_path = os.path.join(save_dir, input_csv) if save_dir else input_csv

    # Чтение CSV-файла
    try:
        df = pd.read_csv(input_path)
        print(f"Файл {input_path} успешно прочитан.")
    except FileNotFoundError:
        print(f"Файл {input_path} не найден.")
        return
    except Exception as e:
        print(f"Ошибка при чтении файла {input_path}: {e}")
        return

    # Проверка наличия необходимых колонок
    required_columns = ["questions", "ground_truth_documents", "filename", "slide_number", "ground_truth"]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Входной файл не содержит следующие необходимые колонки: {missing_columns}")
        return

    responses = []
    r_filenames = []
    r_slide_numbers = []
    contexts_list = []

    # Итерация по строкам с прогрессбаром
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Обработка строк"):
        question = row["questions"]

        if pd.isna(question) or question.strip() == "":
            print(f"Строка {index}: Вопрос пуст. Пропуск.")
            responses.append("")
            r_filenames.append("")
            r_slide_numbers.append("")
            contexts_list.append("")
            continue

        # Получение ответа через RAG пайплайн
        try:
            response_dict = get_response_function(question)
            responses.append(response_dict.get("response", ""))
            r_filenames.append(response_dict.get("r_filename", ""))
            r_slide_numbers.append(response_dict.get("r_slide_number", ""))

            contexts = response_dict.get("contexts", [])
            contexts_list.append(contexts)
        except Exception as e:
            print(f"Строка {index}: Ошибка при получении ответа: {e}")
            responses.append("")
            r_filenames.append("")
            r_slide_numbers.append("")
            contexts_list.append([""])

    # Добавление новых колонок в DataFrame
    df["responses"] = responses
    df["r_filename"] = r_filenames
    df["r_slide_number"] = r_slide_numbers
    df["contexts"] = contexts_list

    # Определение поддиректории и создание её при необходимости
    if sub_dir_name:
        current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
        sub_dir_path = os.path.join(save_dir, f"{sub_dir_name}_{current_datetime}") if save_dir else sub_dir_name
        os.makedirs(sub_dir_path, exist_ok=True)
    else:
        sub_dir_path = save_dir  # Если поддиректория не задана, использовать save_dir


    # Разделение имени файла и расширения
    filename, ext = os.path.splitext(output_csv)

    # Создание нового имени файла с добавлением datetime
    output_filename_with_datetime = output_csv

    # Полный путь к выходному файлу
    output_path = os.path.join(sub_dir_path, output_filename_with_datetime) if sub_dir_path else output_filename_with_datetime

    # Сохранение DataFrame в CSV
    try:
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Результаты успешно сохранены в {output_path}")
    except Exception as e:
        print(f"Ошибка при записи файла CSV: {e}")

    return df

In [ ]:
eval_main_dir = "drive/MyDrive/HacksAI/international24/Evaluate"

updated_df = add_rag_responses_to_csv(
    input_csv="ground_truth_data.csv",
    output_csv="gt_evaluation.csv",
    get_response_function=get_eval_response,
    save_dir=eval_main_dir,
    sub_dir_name="baseline_qwen_2__4",
)

Файл drive/MyDrive/HacksAI/international24/Evaluate/ground_truth_data.csv успешно прочитан.


Batches: 100%|██████████| 1/1 [00:00<00:00, 56.71it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 56.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 51.65it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 54.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 55.35it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 55.39it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 54.45it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.63it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 54.79it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 55.68it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.18it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 51.59it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.53it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 57.02it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 51.28it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.47it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 55.4

Результаты успешно сохранены в drive/MyDrive/HacksAI/international24/Evaluate/baseline_qwen_2__4_20241110_011911/gt_evaluation.csv


In [ ]:
updated_df

,questions,ground_truth_documents,filename,slide_number,ground_truth,responses,r_filename,r_slide_number,contexts
0,Сколько интернет-пользователей было в мире по ...,В 2023 году количество интернет-пользователей ...,1.pdf,2,"['По итогам 2023 года 5,3 млрд человек или 65,...",Я не знаю ответа на ваш вопрос,None,None,[Conclusion\nKEY LEARNINGS\nThe world of influ...
1,Какой процент выручки от рекламы достигала Met...,"В 2020 году доля выручки от рекламы Meta,◊ кот...",1.pdf,7,['В 2020 году доля выручки от рекламы Meta дос...,"В 2020 году доля выручки от рекламы Meta, кото...",1.pdf,7,[Коммуникационные сервисы: социальные сети и м...
2,Какое число пользователей подписных сервисов п...,"Ожидается, что к 2025 году число пользователей...",1.pdf,14,"['Ожидается, что к 2025 году число пользовател...",К 2025 году число пользователей подписных серв...,1.pdf,14,[26\nВИДЕОСЕРВИСЫ: ОНЛАЙН- КИНОТЕАТРЫ\n4\nК он...
3,Какова мировая выручка в индустрии видеоигр ви...,Крупнейшим сегментом рынка потребительских сер...,1.pdf,25,['Мировая выручка в индустрии видеоигр виртуал...,Мировая выручка в индустрии видеоигр виртуальн...,1.pdf,25,[Технологии виртуальной и дополненной реальнос...
4,What contributed to the growth of PC and conso...,With downloaded/boxed PC growth as a primary c...,3.pdf,8,['With downloaded/boxed PC growth as a primary...,PC and console game revenues grew by +2.6% las...,3.pdf,8,[43%57% Our revenues encompass consumer spendi...
5,Какой процентный спад в среднем квартальном вр...,"As lockdowns ended across the world, players t...",3.pdf,16,['Наблюдается спад в среднем квартальном време...,"Среднее квартальное время игры на ПК, Xbox и P...",3.pdf,16,[-11%-4%-1%-4%-6%-1%+3%+1%-6%1%0%\nQ1 21Q2 21Q...
6,Какой объем мирового рынка инфлюенс-маркетинга...,Объем мирового рынка инфлюенс-маркетинга на 20...,4.pdf,4,['Объем мирового рынка инфлюенс-маркетинга на ...,Объем мирового рынка инфлюенс-маркетинга на 20...,4.pdf,4,[4\nОбъем мирового рынка инфлюенс- маркетинга ...
7,Какова главная цель использования ИИ и машинно...,63% респондентов исследования планируют исполь...,4.pdf,8,['Главная цель использования технологий — это ...,"Согласно исследованию, главная цель использова...",4.pdf,8,[8 63% респондентов исследования планируют исп...
8,Какой процент пользователей TikTok составляет ...,"Исследования показывают, что около 60% пользов...",4.pdf,16,['Около 60% пользователей TikTok — это люди в ...,"Исследования показывают, что около 60% пользов...",4.pdf,16,[16\nГ лава 2 > Инфлюенс- маркетинг в сфере не...
9,Как изменилось количество загрузок в мобильном...,The mobile gaming market in 2023 saw a 10% dec...,69.pdf,5,['Мобильный игровой рынок в 2023 году увидел 1...,В 2023 году мобильные загрузки игр показали сл...,69.pdf,14,[The State of Gaming 2024\nAn analysis of the ...


### Поллинг

In [ ]:
# API CONFIG
API_CONFIG_PATH = os.getenv("API_CONFIG_PATH")
with open(API_CONFIG_PATH, "r", encoding='utf-8') as config_file:
  api_config = yaml.safe_load(config_file)

# Извлечение URL и эндпоинтов из конфигурации
BASE_URL = api_config["api"]["base_url"]
POLL_ENDPOINT = f"{BASE_URL}{api_config['api']['endpoints']['poll']}"
COMPLETE_ENDPOINT = f"{BASE_URL}{api_config['api']['endpoints']['complete']}"

In [ ]:
# Функция для опроса POLL_ENDPOINT
def poll():
    response = requests.get(POLL_ENDPOINT)
    response.raise_for_status()  # Проверка на успешный статус ответа
    return response.json()  # Возвращает JSON-ответ от сервера

# Функция для отправки результата на COMPLETE_ENDPOINT
def complete(request_id, message = "", sources = [], suggestions = []):
    complete_response = requests.post(
        COMPLETE_ENDPOINT,
        json={
            "requestId": request_id,
            "message": message,
            "sources": sources,
            "suggestions": suggestions,

        }
    )
    complete_response.raise_for_status()  # Проверка на успешный статус ответа
    return complete_response.json()

# Заглушка для функции check_s3_poll
def check_s3_poll():
    # Эта функция будет добавлена в будущем
    print("Проверка S3... (заглушка)")
    time.sleep(1)  # Задержка для эмуляции работы функции

# Основная функция для обработки генерации ответа
def generator_poll():
    # Опрос POLL_ENDPOINT
    response_data = poll()

    # Извлечение необходимых данных из ответа
    request_id = response_data.get("requestId")
    question = response_data.get("query")

    # Если получены необходимые данные
    if request_id:
        if not question:  # Если question пустой (пустая строка)
            conversational = response_data.get("history", [])
            related_questions = generate_related_questions(conversational)
            complete(
                request_id=request_id,
                suggestions = related_questions,
            )
        else:
          categories = response_data.get("categories", None)
          space = response_data.get("space", None)
          filename = response_data.get("filename", None)
          input_data = {
              "question": question,
              "categories": categories, #
              "space": space, #
              "filename": filename, #
          }
          model_response = get_chat_response_json(input_data)

          complete(
              message = model_response.get("message", ""),
              request_id=request_id,
              sources = model_response.get("sources", []),
          )


# Бесконечный цикл для выполнения generator_poll и check_s3_poll
while True:
    try:
        # Выполнение generator_poll
        generator_poll()

        # Выполнение check_s3_poll
        check_s3_poll()

        # Задержка перед следующей итерацией
        time.sleep(5)

    except Exception as e:
        print(f"Ошибка при опросе: {e}")
        user_input = input("Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: ")

        # Если введено 'q', выходим из цикла
        if user_input.lower() == 'q':
            print("Завершение работы.")
            break  # Выход из бесконечного цикла

Ошибка при опросе: 404 Client Error: Not Found for url: http://158.160.68.33:3001/api/search/poll
Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: 
Ошибка при опросе: 404 Client Error: Not Found for url: http://158.160.68.33:3001/api/search/poll
Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: 
Ошибка при опросе: 404 Client Error: Not Found for url: http://158.160.68.33:3001/api/search/poll
Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: 
Ошибка при опросе: 404 Client Error: Not Found for url: http://158.160.68.33:3001/api/search/poll
Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: 


Batches: 100%|██████████| 1/1 [00:00<00:00, 47.47it/s]


Проверка S3... (заглушка)
Ошибка при опросе: 404 Client Error: Not Found for url: http://158.160.68.33:3001/api/search/poll
Нажмите любую клавишу, чтобы продолжить, или введите 'q' для выхода: q
Завершение работы.
